In [1]:
#import package
#an example code to train the model based on a small synthetic dataset -- with no confounders
import pandas as pd
from collections import defaultdict
import os
import pickle

import scipy
from sklearn.model_selection import KFold

from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, AutoTokenizer
from transformers import BertModel, BertPreTrainedModel, AdamW, BertConfig, LongformerForMaskedLM, LongformerModel
from transformers import get_linear_schedule_with_warmup

from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertPreTrainedModel

from torch.nn import CrossEntropyLoss,MSELoss,BCELoss

import torch
import torch.nn as nn
from scipy.special import softmax
import numpy as np
from scipy.special import logit
from sklearn.linear_model import LogisticRegression
import pandas as pd
from tqdm import tqdm
import math
import gc

gc.collect()

22

In [2]:
#set random seed globally
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    #random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed(123)

Random seed set as 123


In [3]:
# CausalBert v2: Enhanced BERT-based model for causal inference with confounder integration
# This version directly incorporates confounder variables into the neural network architecture

import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd
from collections import defaultdict
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss, BCELoss, MSELoss
from transformers import LongformerModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.linear_model import LogisticRegression
from scipy.special import logit
from tqdm import tqdm

# Global configuration
CUDA = (torch.cuda.device_count() > 0)  # Check if CUDA GPU is available
MASK_IDX = 103  # Token ID for [MASK] token in Longformer tokenizer


def platt_scale(outcome, probs):
    """
    Apply Platt scaling for probability calibration.
    Transforms raw model outputs to well-calibrated probabilities using logistic regression.
    
    Args:
        outcome: Ground truth binary outcomes [n_samples]
        probs: Predicted probabilities to be calibrated [n_samples]
    
    Returns:
        Calibrated probabilities using fitted logistic regression
    """
    logits = logit(probs)
    logits = logits.reshape(-1, 1)
    log_reg = LogisticRegression(penalty='none', warm_start=True, solver='lbfgs')
    log_reg.fit(logits, outcome)
    return log_reg.predict_proba(logits)


def gelu(x):
    """
    Gaussian Error Linear Unit (GELU) activation function.
    Smoother alternative to ReLU that provides better gradient flow.
    
    Args:
        x: Input tensor
    
    Returns:
        GELU-activated tensor: 0.5 * x * (1 + erf(x/√2))
    """
    return 0.5 * x * (1.0 + torch.erf(x / math.sqrt(2.0)))


def make_bow_vector(ids, vocab_size, use_counts=False):
    """
    Convert dense token IDs to sparse bag-of-words representation.
    Useful for creating categorical feature representations from token sequences.
    
    Args:
        ids: torch.LongTensor [batch, features] - Dense tensor of token IDs
        vocab_size: Size of vocabulary for creating BOW vector
        use_counts: If True, use token frequency counts; if False, use binary indicators
    
    Returns:
        Sparse bag-of-words representation [batch, vocab_size]
    """
    vec = torch.zeros(ids.shape[0], vocab_size)
    ones = torch.ones_like(ids, dtype=torch.float)
    
    # Move tensors to GPU if available
    if CUDA:
        vec = vec.cuda()
        ones = ones.cuda()
        ids = ids.cuda()

    # Create BOW vector using scatter_add for efficient sparse operations
    vec.scatter_add_(1, ids, ones)
    vec[:, 1] = 0.0  # Zero out padding tokens (assuming pad token ID is 1)
    
    # Convert to binary indicators if not using counts
    if not use_counts:
        vec = (vec != 0).float()
    
    return vec


class CausalBert(LongformerModel):
    """
    CausalBert v2: Enhanced Longformer-based model for causal inference with confounder integration.
    
    Key improvements over v1:
    1. Direct integration of confounder variables into neural network layers
    2. Enhanced architecture that combines text representations with structured confounders
    3. Improved handling of selection bias through explicit confounder modeling
    
    The model learns:
    1. Rich text representations using pretrained Longformer
    2. Propensity scores considering both text and confounders
    3. Potential outcomes under different treatments (Q-learning) with confounder adjustment
    4. Masked language modeling as auxiliary task for representation learning
    """
    
    def __init__(self, config):
        super().__init__(config)

        # load longformer from input configs
        self.encoder = LongformerModel(config)
        
        self.num_labels = config.num_labels
        self.vocab_size = config.vocab_size
        
        # Masked Language Modeling components
        # These layers project hidden states back to vocabulary space for MLM loss
        self.vocab_transform = nn.Linear(config.hidden_size, config.hidden_size)
        self.vocab_layer_norm = nn.LayerNorm(config.hidden_size)
        self.vocab_projector = nn.Linear(config.hidden_size, config.vocab_size)

        # Q-learning heads: Predict potential outcomes for each treatment
        # KEY CHANGE: Input size is now hidden_size + 2 to accommodate confounders
        # The +2 suggests confounders are 2-dimensional (e.g., [age, income] or [feature1, feature2])
        self.Q_cls = nn.ModuleDict()
        for T in range(2):  # Binary treatment (T=0, T=1)
            self.Q_cls['%d' % T] = nn.Sequential(
                nn.Linear(config.hidden_size + 0, 200),  # Text features + 2 confounder dimensions (uses can change the number of additional confounders)
                nn.ReLU(),
                nn.Linear(200, 1)  # Single output for outcome prediction
            )

        # Propensity score network: Predict P(T=1|text,confounders)
        # Also takes concatenated input of text representations and confounders
        self.g_cls = nn.Sequential(
            nn.Linear(config.hidden_size + 0, 1),  # Text + confounder features
            nn.Sigmoid()  # Output probability between 0 and 1
        )

        self.init_weights()

    def forward(self, W_ids, W_len, W_mask, C, T, Y=None, use_mlm=True):
        """
        Forward pass of CausalBert with confounder integration.
        
        Args:
            W_ids: Token IDs for input text [batch_size, seq_len]
            W_len: Actual lengths of sequences [batch_size]
            W_mask: Attention mask [batch_size, seq_len]
            C: Confounder variables [batch_size, 2] - structured features like age, income, etc.
            T: Treatment indicators [batch_size] - binary treatment assignment
            Y: Outcomes [batch_size] (optional, for training)
            use_mlm: Whether to use masked language modeling auxiliary task
        
        Returns:
            g: Propensity scores [batch_size, 1]
            Q0: Predicted outcomes under treatment 0 [batch_size, 1]
            Q1: Predicted outcomes under treatment 1 [batch_size, 1]
            g_loss: Propensity score prediction loss
            Q_loss: Q-learning outcome prediction loss
            mlm_loss: Masked language modeling loss
        """
        
        # Masked Language Modeling setup for auxiliary learning
        if use_mlm:
            W_len = W_len.unsqueeze(1) - 2  # Subtract 2 for [CLS] and [SEP] special tokens
            mask_class = torch.cuda.FloatTensor if CUDA else torch.FloatTensor
            
            # Randomly select positions to mask (avoiding [CLS] token at position 0)
            mask = (mask_class(W_len.shape).uniform_() * W_len.float()).long() + 1
            target_words = torch.gather(W_ids, 1, mask)
            
            # Create MLM labels: -100 for positions we don't want to predict
            mlm_labels = torch.ones(W_ids.shape).long() * -100
            if CUDA:
                mlm_labels = mlm_labels.cuda()
            
            # Set target labels for masked positions and apply masks to input
            mlm_labels.scatter_(1, mask, target_words)
            W_ids.scatter_(1, mask, MASK_IDX)
        
        # Get text representations from Longformer
        # WARNING: Using self.distilbert instead of inherited model - this ignores pretrained weights!
        outputs = self.encoder(W_ids, attention_mask=W_mask)
        seq_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
        pooled_output = outputs.pooler_output    # [batch_size, hidden_size]
        
        # Masked Language Modeling loss calculation
        if use_mlm:
            prediction_logits = self.vocab_transform(seq_output)
            prediction_logits = gelu(prediction_logits)
            prediction_logits = self.vocab_layer_norm(prediction_logits)
            prediction_logits = self.vocab_projector(prediction_logits)
            
            mlm_loss = CrossEntropyLoss()(
                prediction_logits.view(-1, self.vocab_size), 
                mlm_labels.view(-1)
            )
        else:
            mlm_loss = 0.0
        
        # CONFOUNDER INTEGRATION: Key innovation of this version
        # Ensure confounders have compatible data type with text representations
        C = C.to(pooled_output.dtype)
        
        # Concatenate text representations with confounder variables
        # This creates a joint representation: [text_features, confounder1, confounder2]
        inputs = pooled_output  # no confounders
        #torch.cat((pooled_output, C), 1)  # [batch_size, hidden_size + 2]
        
        # Propensity score prediction using combined text+confounder features
        g = self.g_cls(inputs)
        if Y is not None:  
            # Binary cross-entropy loss for propensity score
            g_loss = BCELoss()(g.flatten(), T.flatten().float())
        else:
            g_loss = 0.0

        # Q-learning: Predict potential outcomes using text+confounder features
        Q_logits_T0 = self.Q_cls['0'](inputs)  # Outcome if treated with T=0
        Q_logits_T1 = self.Q_cls['1'](inputs)  # Outcome if treated with T=1

        # Q-learning loss: Only compute loss for observed treatment-outcome pairs
        # This implements the fundamental Q-learning principle: learn from observed data
        if Y is not None:
            Y = Y.float()
            T0_indices = (T == 0).nonzero().squeeze()  # Samples that received treatment 0
            T1_indices = (T == 1).nonzero().squeeze()  # Samples that received treatment 1
            
            # Compute outcome prediction loss only for corresponding treatment groups
            # This prevents the model from learning incorrect treatment-outcome associations
            if T1_indices.nelement() != 0:
                Q_loss_T1 = MSELoss()(
                    Q_logits_T1.flatten()[T1_indices], 
                    Y.float()[T1_indices]
                )
            else:
                Q_loss_T1 = 0.0
                
            if T0_indices.nelement() != 0:
                Q_loss_T0 = MSELoss()(
                    Q_logits_T0.flatten()[T0_indices], 
                    Y.float()[T0_indices]
                )
            else:
                Q_loss_T0 = 0.0

            Q_loss = Q_loss_T0 + Q_loss_T1
        else:
            Q_loss = 0.0

        Q0 = Q_logits_T0
        Q1 = Q_logits_T1
        
        return g, Q0, Q1, g_loss, Q_loss, mlm_loss


class CausalBertWrapper:
    """
    Enhanced wrapper class for training and inference with CausalBert v2.
    Handles confounder preprocessing and integration with text data.
    """

    def __init__(self, g_weight=1.0, Q_weight=1.0, mlm_weight=1.0, batch_size=32):
        """
        Initialize CausalBert wrapper with multi-task loss weighting.
        
        Args:
            g_weight: Weight for propensity score loss (higher = more focus on treatment prediction)
            Q_weight: Weight for Q-learning loss (higher = more focus on outcome prediction)
            mlm_weight: Weight for masked language modeling loss (higher = more language understanding)
            batch_size: Batch size for training and inference
        """
        # Load pretrained Longformer and extend with causal inference capabilities
        self.model = CausalBert.from_pretrained(
            "allenai/longformer-base-4096",
            num_labels=2,  # Binary treatment
            output_attentions=False,  # Save memory
            output_hidden_states=False  # Save memory
        )
        
        # Move model to GPU if available
        if CUDA:
            self.model = self.model.cuda()

        # Multi-task learning loss weights
        self.loss_weights = {
            'g': g_weight,      # Propensity score learning
            'Q': Q_weight,      # Outcome prediction learning
            'mlm': mlm_weight   # Language understanding preservation
        }
        self.batch_size = batch_size

    def train(self, texts, confounds, treatments, outcomes,
              learning_rate=2e-5, epochs=3, filename=None):
        """
        Train the CausalBert model with integrated confounder variables.
        
        Args:
            texts: List of text documents (e.g., medical records, reviews, patents)
            confounds: List/array of confounder variables [n_samples, 2] (e.g., [age, income])
            treatments: List of treatment indicators (0 or 1)
            outcomes: List of outcome values (continuous or binary)
            learning_rate: Learning rate for AdamW optimizer
            epochs: Number of training epochs
            filename: Optional CSV filename to save training loss progression
        
        Returns:
            Trained model
        """
        dataloader = self.build_dataloader(texts, confounds, treatments, outcomes)

        self.model.train()
        
        # Setup AdamW optimizer with learning rate scheduling
        optimizer = AdamW(self.model.parameters(), lr=learning_rate, eps=1e-8)
        total_steps = len(dataloader) * epochs
        warmup_steps = total_steps * 0.1  # 10% warmup
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=warmup_steps, 
            num_training_steps=total_steps
        )
        
        avg_loss = []
        
        # Training loop with multi-task loss optimization
        for epoch in range(epochs):
            losses = []
            self.model.train()
            
            for step, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
                # Move batch to GPU if available
                if CUDA: 
                    batch = tuple(x.cuda() for x in batch)
                
                W_ids, W_len, W_mask, C, T, Y = batch
                
                # Forward pass with multi-task learning
                self.model.zero_grad()
                g, Q0, Q1, g_loss, Q_loss, mlm_loss = self.model(
                    W_ids, W_len, W_mask, C, T, Y
                )
                
                # Weighted combination of all losses
                total_loss = (self.loss_weights['g'] * g_loss + 
                             self.loss_weights['Q'] * Q_loss + 
                             self.loss_weights['mlm'] * mlm_loss)
 
                # Backward pass and optimization step
                total_loss.backward()
                optimizer.step()
                scheduler.step()
                
                # Track Q-learning loss specifically (main task)
                losses.append(Q_loss.detach().cpu().item())
                
            avg_loss.append(np.mean(losses))
        
        # Save training loss progression if filename provided
        if filename:
            loss_df = pd.DataFrame(avg_loss, columns=["loss"])
            loss_df.to_csv(filename, index=False)
            
        return self.model

    def inference(self, texts, confounds, treatment=None, outcome=None):
        """
        Perform inference with trained model to get predictions.
        
        Args:
            texts: List of text documents
            confounds: List/array of confounder variables
            treatment: Optional treatment indicators (for evaluation)
            outcome: Optional outcome values (for evaluation)
        
        Returns:
            Q0s: Predicted outcomes under treatment 0
            Q1s: Predicted outcomes under treatment 1
            Gs: Predicted propensity scores
            Ts: Treatment indicators
        """
        self.model.eval()
        dataloader = self.build_dataloader(
            texts, confounds, 
            treatments=treatment, outcomes=outcome,
            sampler='sequential'  # Deterministic order for inference
        )
        
        # Collect all predictions
        Q0s, Q1s, Ys, Gs, Ts = [], [], [], [], []
        
        with torch.no_grad():  # Disable gradient computation for efficiency
            for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
                if CUDA: 
                    batch = tuple(x.cuda() for x in batch)
                
                W_ids, W_len, W_mask, C, T, Y = batch
                
                # Forward pass without MLM (faster for inference)
                g, Q0, Q1, _, _, _ = self.model(
                    W_ids, W_len, W_mask, C, T, use_mlm=False
                )
                
                # Convert predictions to CPU and collect
                Q0s += Q0.flatten().detach().cpu().numpy().tolist()
                Q1s += Q1.flatten().detach().cpu().numpy().tolist()
                Ys += Y.flatten().detach().cpu().numpy().tolist()
                Gs += g.flatten().detach().cpu().numpy().tolist()
                Ts += T.flatten().detach().cpu().numpy().tolist()
                
        return Q0s, Q1s, Gs, Ts

    def ATE(self, C, W, df, filename, Y=None, T=None, names=None, dates=None, platt_scaling=False):
        """
        Compute Average Treatment Effect (ATE) using the trained model. Note that we no longer claim causality in the newest version of the manual script.

        This function also returns the estimated citations and  gender (group indicator) propensity. The ATE number should only be used in cases with randomized gender (group) assignment.
        
        ATE represents the average difference in outcomes if everyone received treatment
        vs. if everyone received control, accounting for confounders and selection bias.
        
        Args:
            C: Confounder variables
            W: Text data
            df: DataFrame to store detailed results
            filename: Output CSV filename for results
            Y: True outcomes (optional)
            T: True treatments (optional)
            names: Optional sample names/IDs
            dates: Optional dates
            platt_scaling: Whether to apply Platt scaling to propensity scores
        
        Returns:
            Estimated Average Treatment Effect among the treated (ATT)
        """
        Q0s, Q1s, Gs, Ts = self.inference(W, C, treatment=T, outcome=Y)
        
        # Store comprehensive results in dataframe
        df["Q0"] = Q0s           # Predicted outcome under control (T=0)
        df["Q1"] = Q1s           # Predicted outcome under treatment (T=1)
        df["Gs"] = Gs            # Propensity scores P(T=1|X,C)
        df["true_out"] = Y       # True observed outcomes
        df["T"] = T              # True treatment assignments
        
        # Save results for further analysis
        df[["patent_id", "Q0", "Q1", "Gs", "true_out", "T"]].to_csv(filename, index=False)
        
        # Compute Average Treatment Effect on the Treated (ATT)
        # ATT = E[Q1 - Q0 | T=1] - effect among those who actually received treatment
        treatment_effect = np.array(Q1s) - np.array(Q0s)  # Individual treatment effects
        treated_mask = np.array(Ts)  # Binary mask for treated individuals
        
        # Weighted average of treatment effects among treated individuals
        att = np.mean(np.multiply(treatment_effect, treated_mask)) / np.sum(treated_mask)
        
        return att
    
    def build_dataloader(self, texts, confounds, treatments=None, outcomes=None,
                        tokenizer=None, sampler='random'):
        """
        Build PyTorch DataLoader with proper confounder preprocessing.
        
        Args:
            texts: List of text documents
            confounds: Confounder variables (will be converted to numpy array)
            treatments: Treatment indicators (optional, filled with -1 if None)
            outcomes: Outcome values (optional, filled with -1 if None)
            tokenizer: Pretrained tokenizer (optional)
            sampler: 'random' or 'sequential' data sampling
        
        Returns:
            PyTorch DataLoader with batched, tokenized data
        """
        # Fill missing values with dummy data for inference
        if treatments is None:
            treatments = [-1 for _ in range(len(confounds))]
        if outcomes is None:
            outcomes = [-1 for _ in range(len(treatments))]

        # Load Longformer tokenizer if not provided
        if tokenizer is None:
            tokenizer = AutoTokenizer.from_pretrained(
                'allenai/longformer-base-4096', do_lower_case=True
            )

        # Ensure confounders are numpy array for consistent processing
        confounds = np.asarray(confounds)
        
        # Process each sample: tokenize text and collect confounders
        out = defaultdict(list)
        for i, (W, C, T, Y) in enumerate(zip(texts, confounds, treatments, outcomes)):
            # Tokenize text with proper truncation and padding
            encoded_sent = tokenizer.encode_plus(
                W, 
                add_special_tokens=True,  # Add [CLS] and [SEP] tokens
                max_length=512,           # Longformer context limit
                truncation=True,          # Truncate if longer
                pad_to_max_length=True    # Pad if shorter
            )

            # Store tokenized text data
            out['W_ids'].append(encoded_sent['input_ids'])
            out['W_mask'].append(encoded_sent['attention_mask'])
            out['W_len'].append(sum(encoded_sent['attention_mask']))  # Actual sequence length
            
            # Store targets and confounders
            out['Y'].append(Y)  # Outcomes
            out['T'].append(T)  # Treatments
            out['C'].append(C)  # Confounder variables (should be 2-dimensional)

        # Convert to PyTorch tensors
        data = tuple(torch.tensor(out[x]) for x in ['W_ids', 'W_len', 'W_mask', 'C', 'T', 'Y'])
        dataset = TensorDataset(*data)
        
        # Choose appropriate sampler
        sampler = RandomSampler(dataset) if sampler == 'random' else SequentialSampler(dataset)
        dataloader = DataLoader(dataset, sampler=sampler, batch_size=self.batch_size)

        return dataloader

In [4]:
#set max GPU usage to avoid system crashing
torch.cuda.set_per_process_memory_fraction(1.0, 0)
#clear GPU cache
torch.cuda.empty_cache()

In [5]:
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
total_memory = torch.cuda.get_device_properties(0).total_memory
print('Total:   ', round(total_memory/1024**3,1), 'GB')

Allocated: 0.0 GB
Cached:    0.0 GB
Total:    31.9 GB


C:\Anaconda3\lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [6]:
df=pd.read_parquet("../data/synthetic_1k_sample.parquet")

In [7]:
df

,patent_id,syn_T,true_out_x,T,true_out_y,description_text
0,4180115,False,-6.026118,1,2.484907,DESCRIPTION OF THE PREFERRED EMBODIMENT \nRefe...
1,5485232,True,-4.777419,0,2.397895,DETAILED DESCRIPTION OF THE PREFERRED EMBODIME...
2,6667803,True,-4.646103,0,3.850148,DETAILED DESCRIPTION OF THE PREFERRED EMBODIME...
3,8620805,False,-5.259564,0,2.302585,DETAILED DESCRIPTION\n\nReference will now be ...
4,6372108,False,-5.627249,0,1.098612,EXAMPLES \n\n Example 1 (Comparative Example) ...
...,...,...,...,...,...,...
995,7955450,True,-4.718397,0,0.693147,DETAILED DESCRIPTION OF PREFERRED EMBODIMENTS\...
996,8030783,False,-5.663879,0,0.693147,BEST MODE FOR CARRYING OUT THE INVENTION\n\nRe...
997,4371475,False,-5.414377,0,1.098612,The present invention will be further illustra...
998,6747008,False,-5.342205,1,3.332205,DETAILED DESCRIPTION OF THE INVENTION \n\n All...


In [ ]:
if __name__ == '__main__':
    import pandas as pd
#run the model fitting code to get the text quality, group propensity, and model. the df should include text, group indicator (gender), and target (citation counts)
    cb = CausalBertWrapper(batch_size=16,  
        g_weight=1, Q_weight=1, mlm_weight=1)
#no confounders
    confounders=pd.Series(np.zeros(df.shape[0]),name="C")
#fit the model
    cb.train(df['description_text'], confounders, df['syn_T'], df['true_out_x'], epochs=20,filename="../output/small_syn_test_loss.csv")
#get the model outputs (only use ATE numbers when group indicators are randomly assigned)
    cb.ATE(confounders, df['description_text'],df,"../output/small_syn_test.csv",Y=df['true_out_x'],T=df['syn_T'] ,platt_scaling=True)
#save model
    file_name = '../output/small_syn_test_model.obj'
    with open(file_name, 'wb') as file:
        pickle.dump(cb.model, file)
        print(f'Object successfully saved to "{file_name}"')

In [ ]:
#check the results

In [11]:
dfres=pd.read_csv("../output/small_syn_test.csv")

In [12]:
dfres

,patent_id,Q0,Q1,Gs,true_out,T
0,4180115,-5.457075,-4.433169,0.483335,-6.026118,False
1,5485232,-5.457129,-4.433617,0.483348,-4.777419,True
2,6667803,-5.455376,-4.432090,0.483371,-4.646103,True
3,8620805,-5.454805,-4.431591,0.483738,-5.259564,False
4,6372108,-5.454700,-4.431559,0.483336,-5.627249,False
...,...,...,...,...,...,...
995,7955450,-5.454932,-4.431803,0.483401,-4.718397,True
996,8030783,-5.454392,-4.431137,0.483559,-5.663879,False
997,4371475,-5.456200,-4.432624,0.483277,-5.414377,False
998,6747008,-5.451680,-4.429364,0.483392,-5.342205,False


In [16]:
print(f'The gap (ATE since group indicators are randomly assigned) is {np.mean(dfres[dfres["T"]==1].Q1)-np.mean(dfres[dfres["T"]==0].Q0)}')
print('The true gap is 1.')

The gap (ATE since group indicators are randomly assigned) is 1.0235336682078016
The true gap is 1.
